<a href="https://colab.research.google.com/github/thavha/Regression-predict-group-5/blob/master/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Preprocessing**
Importing Liabraries


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

**Importing the data set** and checking the data

In [0]:
dataset = pd.read_csv('Train.csv')
###x =dataset.iloc[:,:-1].values
###y = dataset.iloc[:,-1].values

In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   User Id                                    21201 non-null  object 
 2   Vehicle Type                               21201 non-null  object 
 3   Platform Type                              21201 non-null  int64  
 4   Personal or Business                       21201 non-null  object 
 5   Placement - Day of Month                   21201 non-null  int64  
 6   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 7   Placement - Time                           21201 non-null  object 
 8   Confirmation - Day of Month                21201 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 10  Confirmation - Time   

selecting only the rlevent  columns that will be used for the model and replacing white space with _ on columns names

In [0]:
new_df = dataset[['User Id','Platform Type','Personal or Business','Placement - Day of Month','Placement - Weekday (Mo = 1)','Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Arrival at Destination - Time','Distance (KM)','Temperature','Rider Id','Time from Pickup to Arrival']].copy()
new_df.columns = [col.replace(" ","_") for col in new_df.columns] 


In [21]:
new_df.head()

,User_Id,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival
0,User_Id_633,3,Business,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:39:55 AM,4,20.4,Rider_Id_432,745
1,User_Id_2285,3,Personal,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993
2,User_Id_265,3,Business,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,1:00:38 PM,3,NaN,Rider_Id_155,455
3,User_Id_1402,3,Business,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341
4,User_Id_1737,1,Personal,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214


encoding personal or business column


In [22]:
new_df = pd.get_dummies(new_df, columns=['Personal_or_Business'], prefix = ['personal'])
new_df = new_df.iloc[:, :-1]
new_df.columns = [*new_df.columns[:-1], 'Business']
new_df.head()

,User_Id,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival,Business
0,User_Id_633,3,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:39:55 AM,4,20.4,Rider_Id_432,745,1
1,User_Id_2285,3,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993,0
2,User_Id_265,3,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,1:00:38 PM,3,NaN,Rider_Id_155,455,1
3,User_Id_1402,3,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341,1
4,User_Id_1737,1,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214,0


dealing with the missing valuse replace with the mean/median temperature 

In [23]:
new_df['Temperature'] = new_df.Temperature.fillna(new_df.Temperature.median())
new_df.head()

,User_Id,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival,Business
0,User_Id_633,3,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:39:55 AM,4,20.4,Rider_Id_432,745,1
1,User_Id_2285,3,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993,0
2,User_Id_265,3,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,1:00:38 PM,3,23.5,Rider_Id_155,455,1
3,User_Id_1402,3,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341,1
4,User_Id_1737,1,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214,0


dealing with time columns

In [0]:
new_df['Placement_-_Time 24'] = (pd.to_datetime(new_df['Placement_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
new_df['Confirmation_-_Time 24'] = (pd.to_datetime(new_df['Confirmation_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
new_df['Arrival_at_Pickup_-_Time 24'] = (pd.to_datetime(new_df['Arrival_at_Pickup_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))
new_df['Arrival_at_Destination_-_Time 24'] = (pd.to_datetime(new_df['Arrival_at_Destination_-_Time'].str.strip(), format='%I:%M:%S %p')
                  .dt.strftime('%H:%M'))

In [29]:
new_df.head()

,User_Id,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival,Business,Placement_-_Time 24,Confirmation_-_Time 24,Arrival_at_Pickup_-_Time 24,Arrival_at_Destination_-_Time 24
0,User_Id_633,3,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:39:55 AM,4,20.4,Rider_Id_432,745,1,09:35,09:40,10:04,10:39
1,User_Id_2285,3,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993,0,11:16,11:23,11:40,12:17
2,User_Id_265,3,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,1:00:38 PM,3,23.5,Rider_Id_155,455,1,12:39,12:42,12:49,13:00
3,User_Id_1402,3,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341,1,09:25,09:26,09:37,10:05
4,User_Id_1737,1,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214,0,09:55,09:56,10:03,10:25


In [30]:
new_df.drop(['Placement_-_Time', 'Confirmation_-_Time','Arrival_at_Pickup_-_Time','Arrival_at_Destination_-_Time'], axis=1)

,User_Id,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival,Business,Placement_-_Time 24,Confirmation_-_Time 24,Arrival_at_Pickup_-_Time 24,Arrival_at_Destination_-_Time 24
0,User_Id_633,3,9,5,4,20.4,Rider_Id_432,745,1,09:35,09:40,10:04,10:39
1,User_Id_2285,3,12,5,16,26.4,Rider_Id_856,1993,0,11:16,11:23,11:40,12:17
2,User_Id_265,3,30,2,3,23.5,Rider_Id_155,455,1,12:39,12:42,12:49,13:00
3,User_Id_1402,3,15,5,9,19.2,Rider_Id_855,1341,1,09:25,09:26,09:37,10:05
4,User_Id_1737,1,13,1,9,15.4,Rider_Id_770,1214,0,09:55,09:56,10:03,10:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,User_Id_2001,3,20,3,3,28.6,Rider_Id_953,9,0,15:54,15:55,15:58,16:20
21197,User_Id_1796,3,13,6,7,26.0,Rider_Id_155,770,1,10:13,10:13,10:20,10:46
21198,User_Id_2956,3,7,4,20,29.2,Rider_Id_697,2953,1,17:06,17:07,17:30,18:40
21199,User_Id_2524,1,4,3,13,15.0,Rider_Id_347,1380,0,09:31,09:31,09:38,10:08
